In [ ]:
import datetime as dt
import pathlib

import geopandas as gpd
import google.auth
import numpy as np
import pandas as pd
from gtfs_utils import *
from gtfslite import GTFS
from retrospective_feed_generation import _filter_non_rt_trips, _filter_na_stop_times
from retrospective_feed_generation import *
from shared_utils import catalog_utils, gtfs_utils_v2, rt_dates
from warehouse_utils import *

COLUMN_MAP = {
    "schedule_column": "scheduled_arrival_sec",
    "rt_column": "rt_arrival_sec",
    "stop_sequence_column": "stop_sequence",
    "trip_instance_key_column": "trip_instance_key",
    "trip_id_column": "trip_id",
    "stop_id_column": "stop_id",
    "nonmonotonic_column": "flag_nonmonotonic_arrival_sec",
}

COLUMN_MAP_IMPUTED = {
    **COLUMN_MAP,
    "rt_column": "imputed_arrival_sec",
}

### Get RT Data

In [ ]:
CREDENTIALS, _ = google.auth.default()

In [ ]:
SAMPLE_DATE_STR = rt_dates.DATES["apr2025"]
FEED_NAME = "Big Blue Bus Schedule"

In [ ]:
SAMPLE_DATE_STR

In [ ]:
# gtfs_dataset_key = (
#    gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(
#        selected_date=SAMPLE_DATE_STR, keep_cols=["name", "gtfs_dataset_key"]
#    )
#    .set_index("name")
#    .at[FEED_NAME, "gtfs_dataset_key"]
# )
# gtfs_dataset_key

In [ ]:
# schedule_rt_stop_times_single_agency = get_schedule_rt_stop_times_table(
#    gtfs_dataset_key,
#    SAMPLE_DATE_STR
# )

In [ ]:
# schedule_rt_stop_times_single_agency.to_parquet("test.parquet")

In [ ]:
schedule_rt_stop_times_single_agency = _filter_non_rt_trips(
    pd.read_parquet("test.parquet"),
    **COLUMN_MAP
)

In [ ]:
flagged_stop_times = add_monotonic_flag_to_df(
    schedule_rt_stop_times_single_agency, **COLUMN_MAP
)

In [ ]:
imputed_stop_times = impute_first_last(flagged_stop_times, **COLUMN_MAP).copy()

In [ ]:
imputed_stop_times["nonmonotonic_imputed_sec"] = impute_non_monotonic_rt_times(
    imputed_stop_times, **COLUMN_MAP_IMPUTED
)

In [ ]:
imputed_stop_times["nonmonotonic_imputed_sec"].notna().all()

In [ ]:
imputed_stop_times["gap_imputed_sec"] = impute_short_gaps(
    imputed_stop_times, max_gap_length=5, **{**COLUMN_MAP_IMPUTED, "rt_column": "nonmonotonic_imputed_sec"}
)

In [ ]:
imputed_stop_times = _filter_na_stop_times(imputed_stop_times, **{**COLUMN_MAP_IMPUTED, "rt_column": "gap_imputed_sec"})

In [ ]:
imputed_stop_times.loc[imputed_stop_times["gap_imputed_sec"].isna()]

### Get schedule feed

In [ ]:
# TODO: right now this was just a download based on the url in airtable
# Need to make it traceable instead
GTFS_FEED_PARENT = "../conveyal_update/feeds_2025-04-16/socal/Big_Blue_Bus_Schedule_7a3f513c343b16a30c135ed7d332b6d6_gtfs.zip/"
GTFS_FEED_GLOB = "*.zip"

# GTFS_FEED_PARENT = "./feeds/"
# GTFS_FEED_GLOB = "big_blue_bus_2025-03*.zip"

ARBITRARY_SERVICE_ID = "0"
GTFS_DATE_STRFTIME_CODE = "%Y%m%d"

In [ ]:
feed_paths = pathlib.Path(GTFS_FEED_PARENT).glob(GTFS_FEED_GLOB)
feed_path = next(feed_paths)
assert next(feed_paths, None) is None, "Ambiguous Schedule Feed"
feed = GTFS.load_zip(feed_path)
feed_filtered = subset_schedule_feed_to_one_date(
    feed, dt.date.fromisoformat(SAMPLE_DATE_STR)
)

### Merge schedule / rt

In [ ]:
output_feed = make_retrospective_feed_single_date(
    filtered_input_feed=feed_filtered,
    stop_times_table=imputed_stop_times,
    stop_times_desired_columns=[
        "trip_id",
        "arrival_time",
        "departure_time" "drop_off_type",
        "pickup_type",
        "stop_headsign",
        "stop_id",
        "stop_sequence",
    ],
    **{**COLUMN_MAP_IMPUTED, "rt_column": "gap_imputed_sec"}
)

In [ ]:
output_feed.write_zip(f"output_feeds/bbb_test_{SAMPLE_DATE_STR}.zip")

### Dropped shapes and stops

In [ ]:
print("Get dropped shapes by their frequency")
feed_filtered.trips.loc[
    ~feed_filtered.trips.shape_id.isin(output_feed.trips.shape_id.unique()), "shape_id"
].value_counts()

In [ ]:
print("Get dropped stops by the number of trips serving them in the original feed")
pd.DataFrame(
    feed_filtered.stop_times.loc[
        ~feed_filtered.stop_times.stop_id.isin(output_feed.stop_times.stop_id.unique()),
        "stop_id",
    ]
    .value_counts()
    .rename("stop_count")
).merge(
    feed_filtered.stops.set_index("stop_id")["stop_name"],
    how="left",
    left_index=True,
    right_index=True,
).head()

### Sample Trip

In [ ]:
imputed_stop_times.loc[imputed_stop_times.trip_id == "143110"]

In [ ]:
output_feed.stop_times.loc[output_feed.stop_times["trip_id"] == "143110"]

In [ ]:
imputed_stop_times = imputed_stop_times

In [ ]:
feed_filtered.stop_times.loc[
    feed_filtered.stop_times["trip_id"] == "902110"
].sort_values("stop_sequence")